## Pixels per class

This notebook counts the pixels per class in the three rasters generated by the iceplant classification model. 
The three rasters are non-overlapping and they cover the Santa Barbara County coast. These are:

* merged results of NAIP scenes facing south with crs 26910 (raster 0),
* merged results of NAIP scenes facing west with crs 26910 (raster 1),
* merged results of NAIP scenes with crs 26911 (raster 2).
    
 

In [1]:
import os
import rioxarray as rioxr
import numpy as np
import pandas as pd

In [2]:
year = 2020
prefix = 'salt13_p30'
filtering = False

In [3]:
# -----------------------------------------------
# Load rasters
root = '/home/jovyan/msai4earth-esa/iceplant_detection/processing_results/' + prefix +'_'
if filtering:
    root += 'filter_'
root += 'clip_preds_' + str(year)

fp = os.path.join(root, prefix+'_merged_crs26910_S_'+str(year)+'.tif')
r_26910_S = rioxr.open_rasterio(fp).squeeze()

fp = os.path.join(root, prefix+'_merged_crs26910_W_'+str(year)+'.tif')
r_26910_W = rioxr.open_rasterio(fp).squeeze()

fp = os.path.join(root, prefix+'_merged_crs26911_'+str(year)+'.tif')
r_26911 = rioxr.open_rasterio(fp).squeeze()

rasters = [r_26910_S, r_26910_W, r_26911]

# -----------------------------------------------
# count total number of pixels per category in combined rasters
n_nonice = []
n_ice = []
n_ground = []
n_water = []

for raster in rasters:
    n_nonice.append(len(np.where(raster == 0)[0]))
    n_ice.append(len(np.where(raster == 1)[0]))
    n_ground.append(len(np.where(raster == 2)[0]))
    n_water.append(len(np.where(raster == 3)[0]))
    

# -----------------------------------------------
rasters = [prefix+'_merged_crs26910_S_'+str(year),
                prefix+'_merged_crs26910_W_'+str(year), 
                prefix+'_merged_crs26911_'+str(year)]

In [3]:
# -----------------------------------------------
# assemble and save dataframe with pixel counts
D = { 'n_other_veg': n_nonice,
      'n_ice': n_ice,
      'n_ground': n_ground, 
      'n_water': n_water}

df = pd.DataFrame(D)
df['raster'] = rasters

df.to_csv(prefix+'_rasters_'+str(year)+'_pixel_counts.csv', index=False)

# -----------------------------------------------
df

In [5]:
# -----------------------------------------------
# assemble and save dataframe with pixel counts, 
# water and ground are combined into same class

n_non_veg = [x+y for x,y in zip(n_ground,n_water)]

combined_df = pd.DataFrame({'n_other_veg': n_nonice,
      'n_ice': n_ice,
      'n_nonveg': n_non_veg})

combined_df['raster'] = rasters

combined_df.to_csv(prefix+'_rasters_'+str(year)+'_combined_pixel_counts.csv', index=False)

# -----------------------------------------------
combined_df

,n_other_veg,n_ice,n_nonveg,raster
0,33875887,5308601,114694154,salt13_p30_merged_crs26910_S_2020
1,499575,8816,1871030,salt13_p30_merged_crs26910_W_2020
2,85798004,664006,71506303,salt13_p30_merged_crs26911_2020


In [20]:
total_df = pd.DataFrame([combined_df.sum(numeric_only=True)])
total_df

,n_other_veg,n_ice,n_nonveg
0,120173466,5981423,188071487


In [23]:
total_df.to_csv(prefix+'_total_pixel_counts.csv', index=False)